## DECODING GOOGLE NEWS ARTICLES 


In [1]:
import pandas as pd
#import geopandas as gpd
import json
from ast import literal_eval
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import json
import base64
import requests
import bs4
from bs4 import BeautifulSoup
import re
#import lxml.etree

#pip install geopandas
#pip install spacy
#python -m spacy download en_core_web_sm
#pip install lxml
#pip install parser-libraries 


In [2]:
#rz = pd.read_csv("full_rizz.csv", sep = ";")


# read in the dataset 
df = pd.read_json('animal_news_database.json', lines = True)
# Drop Twitter handles as they are not species names
df = df[df.scientific_name.str.contains('@') == False]
df.reset_index(drop = True, inplace = True)


df = df.drop(['_id'], axis=1)

# Explode the articles column
s = df['articles'].map(dict.values).explode()
# NaN values need to be droppped
s = s.dropna()
# Drop the column articles and join the exploded values to the dataframe
df_full = df.drop(columns=['articles']).join(pd.DataFrame([*s], s.index))

# Processing for articles       
# keep only unique article instances
df_media = df_full.drop_duplicates(subset=['text'])
# Transform date column into DateTime
df_media["date"] = df_media["date"].apply(pd.to_datetime)
# Sort by oldest to newest
df_media = df_media.sort_values(by="date")
# reset index
df_media.reset_index(drop = True, inplace = True)
# Remove any rows with blank data for the text column
df_media = df_media[df_media['text'].notna()]

/tmp/ipykernel_59/2430195414.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_media["date"] = df_media["date"].apply(pd.to_datetime)


In [7]:
"""Decode encoded Google News entry URLs."""
import base64
import functools
import re

# Ref: https://stackoverflow.com/a/59023463/

_ENCODED_URL_PREFIX = "https://news.google.com/articles/"
_ENCODED_URL_RE = re.compile(fr"^{re.escape(_ENCODED_URL_PREFIX)}(?P<encoded_url>[^?]+)")
_DECODED_URL_RE = re.compile(rb'^\x08\x13".+?(?P<primary_url>http[^\xd2]+)\xd2\x01')


@functools.lru_cache(2048)
def _decode_google_news_url(url: str) -> str:
    match = _ENCODED_URL_RE.match(url)
    encoded_text = match.groupdict()["encoded_url"]  # type: ignore
    encoded_text += "==="  # Fix incorrect padding. Ref: https://stackoverflow.com/a/49459036/
    decoded_text = base64.urlsafe_b64decode(encoded_text)

    match = _DECODED_URL_RE.match(decoded_text)
    primary_url = match.groupdict()["primary_url"]  # type: ignore
    primary_url = primary_url.decode()
    return primary_url


def decode_google_news_url(url: str) -> str:  # Not cached because not all Google News URLs are encoded.
    """Return Google News entry URLs after decoding their encoding as applicable."""
    return _decode_google_news_url(url) if url.startswith(_ENCODED_URL_PREFIX) else url

In [8]:
# Create an empty list for the urls
urls = []

# loop through the urls in the column that contains links to googlenews articles
for url in df_media['link']:
    try:
        new = decode_google_news_url(url)
        urls.append(new)
    
    # Attribute error caused by a faulty link will be removed from dataset
    except AttributeError:
        df_media.drop(df_media.loc[df_media.link == url].index, inplace = True)

# After dropping rows we will need to reset the index
df_media = df_media.reset_index(drop=True)

In [12]:
# The lengths of the list and dataframe will be the same, as will their order
# We can concatenate into a new dataframe
df_update = pd.concat([df_media, pd.DataFrame({'url':urls})], axis=1)
# Store the dataframe 
df_update.to_csv("csv/animals_exploded.csv", sep = ';', index = False)

## SHARECOUNT RETRIEVAL HERE

In [2]:
df_update = pd.read_csv("articles-w-full_sharecount.csv", sep = ";")

In [12]:
df_update[10000:]

,common_name,scientific_name,distribution,iucn_status,date_saved,text,title,link,image,date,ner,url,sharecount
10000,Gharial,Gavialis gangeticus,"Bangladesh, Bhutan, India, Myanmar, Nepal, Pak...",CR,2020-06-18,Rastriya Samachar Samiti\n\nShare Now:\n\n\n\n...,Another Nepali succumbs to coronavirus infecti...,https://news.google.com/articles/CBMiYGh0dHBzO...,https://thehimalayantimes.com/wp-content/uploa...,2020-05-05,"[{'sent': 'Rastriya Samachar Samiti\n\n', 'ent...",https://thehimalayantimes.com/nepal/another-ne...,NaN
10001,Luth,Dermochelys coriacea,"Albania, Algeria, American Samoa, Angola, Angu...",VU,2020-06-18,- Some parents abandon their children at the L...,How parents abandon children at LUTH due to de...,https://news.google.com/articles/CBMiUmh0dHBzO...,https://netstorage-legit.akamaized.net/images/...,2020-05-05,[{'sent': '- Some parents abandon their childr...,https://www.legit.ng/1326957-revealed-how-pare...,NaN
10002,Drill,Mandrillus leucophaeus,"Cameroon, Equatorial Guinea, Nigeria",Endangered,2020-06-18,"courtesy ONEswim.com, a SwimmersBest brand\n\n...",Backyard Pool Drill: Resisted Kick Set,https://news.google.com/articles/CBMiO2h0dHBzO...,https://cdn.swimswam.com/wp-content/uploads/20...,2020-05-05,"[{'sent': 'courtesy ONEswim.com, a SwimmersBes...",https://swimswam.com/backyard-pool-drill-resis...,NaN
10003,Mountain Lion,Puma concolor,"Argentina, Belize, Bolivia (Plurinational Stat...",Least Concern,2020-06-18,Each week Rowe Fine Art Gallery is featuring o...,Creature Comforts Livestream Video,https://news.google.com/articles/CBMiS2h0dHBzO...,https://www.signalsaz.com/wp-content/uploads/2...,2020-05-05,[{'sent': 'Each week Rowe Fine Art Gallery is ...,https://www.signalsaz.com/events/events/creatu...,NaN
10004,Vicuna,Vicugna vicugna,"Argentina, Bolivia (Plurinational State of), C...",Least Concern,2020-06-18,Kerala Blasters’ midfield sensation Abdul Saha...,"Eagerly waiting to play under Kibu Vicuna, adm...",https://news.google.com/articles/CBMidmh0dHBzO...,https://cdn-images.spcafe.in/img/es3-cfill-w80...,2020-05-05,[{'sent': 'Kerala Blasters’ midfield sensation...,https://sportscafe.in/football/articles/2020/m...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10687,Grizzly Bear,Ursus arctos,"Afghanistan, Albania, Armenia, Austria, Azerba...",Least Concern,2020-06-18,Volunteer to help with crowds at Wyoming grizz...,Volunteer to help with crowds at Wyoming grizz...,https://news.google.com/articles/CBMiaWh0dHBzO...,https://s.hdnux.com/photos/01/12/22/44/1947472...,2020-05-30,[{'sent': 'Volunteer to help with crowds at Wy...,https://www.michigansthumb.com/news/article/Vo...,NaN
10688,Irrawaddy Dolphin,Orcaella brevirostris,"Australia, Bangladesh, Brunei Darussalam, Camb...",EN,2020-06-18,"In a unique initiative, researchers from the I...",IIT researchers help Chilika Lake in Odisha to...,https://news.google.com/articles/CBMiiwFodHRwc...,https://www.deccanherald.com/sites/dh/files/ar...,2020-05-31,"[{'sent': 'In a unique initiative, researchers...",https://www.deccanherald.com/national/east-and...,NaN
10689,Panther,Panthera pardus,"Afghanistan, Algeria, Angola, Armenia, Azerbai...",VU,2020-06-18,Lonzo Ball is never one to shy away from an op...,"Lonzo Ball jokingly compares JJ Redick, Jrue H...",https://news.google.com/articles/CBMihAFodHRwc...,https://usatlonzowire.files.wordpress.com/2020...,2020-05-31,[{'sent': 'Lonzo Ball is never one to shy away...,https://lonzowire.usatoday.com/2020/05/30/lonz...,NaN
10690,Lion-tailed Macaque,Macaca silenus,India,Endangered,2020-06-18,"During the nation-wide lockdown, most governme...",Flood of approvals by wildlife board sets the ...,https://news.google.com/articles/CBMiemh0dHBzO...,https://www.deccanherald.com/sites/dh/files/ar...,2020-05-31,[{'sent': 'But this nation-wide inactivity has...,https://www.deccanherald.com/specials/insight/...,NaN


In [13]:
# From Sharecount API 
# 10,000 quota per day 
from requests.structures import CaseInsensitiveDict

sharecount = []
for url in df_update['url'][10000:]: 
    
    api = "https://api.sharedcount.com/v1.1?apikey=2a0336186dc8f5740fef9c1eb81947d66119c29f&url=" + url
                                                   
    headers = CaseInsensitiveDict()
    headers["Accept"] = "application/json"
    
    resp = requests.get(api, headers=headers)
    details = resp.json()
    #for n in range(len(details)):
        #count = details[n]['Facebook']['share_count']
        #print(count)
 
    
    fb_count = details['Facebook']['share_count']
    sharecount.append(fb_count)
        
   # except KeyError:
       # print(url, details)
        #rz.drop(rz.loc[rz.url == url].index, inplace = True)


In [14]:
len(sharecount)

692

In [17]:
pd.Series(sharecount).set_axis(range(10000, 10692))

10000     25.0
10001     19.0
10002     15.0
10003      0.0
10004      1.0
         ...  
10687      0.0
10688      0.0
10689      0.0
10690    373.0
10691     31.0
Length: 692, dtype: float64

In [19]:
df_update['sharecount'].iloc[10000:].fillna(pd.Series(sharecount).set_axis(range(10000, 10692)), inplace = True)

In [26]:
df_update

,common_name,scientific_name,distribution,iucn_status,date_saved,text,title,link,image,date,ner,url,sharecount
0,White-winged Wood Duck,Asarcornis scutulata,"Bangladesh, Cambodia, India, Indonesia, Lao Pe...",EN,2020-06-18,"TEMPO.CO, Jakarta - Holding a huge appeal for ...",Lampung Holds Huge Appeal for International To...,https://news.google.com/articles/CBMiU2h0dHA6L...,https://statik.tempo.co/data/2015/02/01/id_366...,1970-01-01,"[{'sent': 'TEMPO.CO, Jakarta - Holding a huge ...",http://en.tempo.co/read/786937/lampung-holds-h...,0.0
1,Javan Rhinoceros,Rhinoceros sondaicus,"Bangladesh, Cambodia, China, India, Indonesia,...",Critically Endangered,2020-06-18,"TEMPO.CO, Jakarta - Director of the Biodiversi...",Javan Rhinos to be Transferred to Cikepuh Wild...,https://news.google.com/articles/CBMiWmh0dHBzO...,https://statik.tempo.co/data/2011/12/29/id_100...,1970-01-01,"[{'sent': 'TEMPO.CO, Jakarta - Director of the...",https://en.tempo.co/read/878302/javan-rhinos-t...,0.0
2,White-winged Wood Duck,Asarcornis scutulata,"Bangladesh, Cambodia, India, Indonesia, Lao Pe...",EN,2020-06-18,"TEMPO.CO, East Lampung - President Joko Widodo...",Jokowi Names Sumatran Rhino `Delilah`,https://news.google.com/articles/CBMiQ2h0dHBzO...,https://statik.tempo.co/data/2015/02/09/id_369...,1970-01-01,"[{'sent': 'TEMPO.CO, East Lampung - President ...",https://en.tempo.co/read/791199/jokowi-names-s...,0.0
3,Javan Gibbon,Hylobates moloch,Indonesia,Endangered,2020-06-18,"TEMPO.CO, Nairobi - The world`s last male nort...",World`s Last Male Northern White Rhino Dies,https://news.google.com/articles/CBMiSmh0dHBzO...,https://statik.tempo.co/data/2017/08/23/id_635...,1970-01-01,"[{'sent': 'TEMPO.CO, Nairobi - The world`s las...",https://en.tempo.co/read/916815/worlds-last-ma...,0.0
4,Javan Gibbon,Hylobates moloch,Indonesia,Endangered,2020-06-18,"TEMPO.CO, Phnom Penh - Wildlife researchers in...",Cambodia Conservationists Find Rare Cache of C...,https://news.google.com/articles/CBMiXGh0dHBzO...,https://statik.tempo.co/data/2014/11/25/id_346...,1970-01-01,"[{'sent': ""TEMPO.CO, Phnom Penh - Wildlife res...",https://www.tempo.co/read/887698/cambodia-cons...,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10687,Grizzly Bear,Ursus arctos,"Afghanistan, Albania, Armenia, Austria, Azerba...",Least Concern,2020-06-18,Volunteer to help with crowds at Wyoming grizz...,Volunteer to help with crowds at Wyoming grizz...,https://news.google.com/articles/CBMiaWh0dHBzO...,https://s.hdnux.com/photos/01/12/22/44/1947472...,2020-05-30,[{'sent': 'Volunteer to help with crowds at Wy...,https://www.michigansthumb.com/news/article/Vo...,0.0
10688,Irrawaddy Dolphin,Orcaella brevirostris,"Australia, Bangladesh, Brunei Darussalam, Camb...",EN,2020-06-18,"In a unique initiative, researchers from the I...",IIT researchers help Chilika Lake in Odisha to...,https://news.google.com/articles/CBMiiwFodHRwc...,https://www.deccanherald.com/sites/dh/files/ar...,2020-05-31,"[{'sent': 'In a unique initiative, researchers...",https://www.deccanherald.com/national/east-and...,0.0
10689,Panther,Panthera pardus,"Afghanistan, Algeria, Angola, Armenia, Azerbai...",VU,2020-06-18,Lonzo Ball is never one to shy away from an op...,"Lonzo Ball jokingly compares JJ Redick, Jrue H...",https://news.google.com/articles/CBMihAFodHRwc...,https://usatlonzowire.files.wordpress.com/2020...,2020-05-31,[{'sent': 'Lonzo Ball is never one to shy away...,https://lonzowire.usatoday.com/2020/05/30/lonz...,0.0
10690,Lion-tailed Macaque,Macaca silenus,India,Endangered,2020-06-18,"During the nation-wide lockdown, most governme...",Flood of approvals by wildlife board sets the ...,https://news.google.com/articles/CBMiemh0dHBzO...,https://www.deccanherald.com/sites/dh/files/ar...,2020-05-31,[{'sent': 'But this nation-wide inactivity has...,https://www.deccanherald.com/specials/insight/...,373.0


In [18]:
all_articles_sharecount = pd.concat([df_update, pd.DataFrame({'sharecount':sharecount})], axis=1) 

In [25]:
df_update.to_csv('csv/articles-w-full_sharecount.csv', sep = '~', index = False)

In [21]:
## Directly retrieveing from Facebook
# Has lower request limit

from requests.structures import CaseInsensitiveDict
import requests

sharecount = []
for url in df_update['url'][:100]:
    headers = CaseInsensitiveDict()
    headers["Accept"] = "application/json"
    
    token = "EAAIxJUBe0zYBAOKUVYhZCHWRlZBm0Pj4M8DGka4LvaFuyeKCMLF8NG2rGtamLjwAMZBSrMFWPY6ZAnpZAihDhP7wjs9drlJ4W8C1ZCCyguI2Kg8rxgfUVrv22sFrcNLXYS1PYZAZALqr2ZB4bLpKMYHC7yGA7lXF8tf3G6BOI6YdQArlLt5bzY6TMZC73HTBr4ztpbRkSefUXGFZB9TTgbyZBL8nY8sSq5YuZAFy9owthwenPOMvK3K9ZACtP5hBiY2GfV4GcZD"
    resp = requests.get(f'https://graph.facebook.com/?id={url}&fields=engagement&access_token={token}', headers=headers).json()
    #print(resp)
    
    #break
    try:
        fb_shares = (resp['engagement']['share_count'])
        sharecount.append(fb_shares)
    except KeyError:
        print("API Limit exceeded")
        
        break
    


API Limit exceeded
